In [77]:
import os
import cv2
import numpy as np
import face_recognition as fr
import pyttsx3
import time

In [78]:
path='Images'
my_list=os.listdir(path)
my_list

['decaprio.jpeg',
 'dq.jpeg',
 'Mammootty.jpg',
 'Mohanlal_Viswanathan_BNC.jpg',
 'Tom_cruse.jpg']

In [79]:
imgs=[]
class_names=[]
for i in my_list:
    img_path=os.path.join(path,i)
    img_r=cv2.imread(img_path)
    imgs.append(img_r)
    class_names.append(i.split('.')[0])
print(class_names)

['decaprio', 'dq', 'Mammootty', 'Mohanlal_Viswanathan_BNC', 'Tom_cruse']


In [80]:
def encode_face(images):
    encoded_list=[]
    for i in imgs:
        imgg=cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        face_loc=fr.face_locations(imgg)
        if len(face_loc)==0:
            print("⚠️ No face found in one image — skipping")
            continue
        face_encode=fr.face_encodings(imgg,face_loc)[0]
        encoded_list.append(face_encode)
    return encoded_list

In [81]:
encoded_known_face=encode_face(imgs)
encoded_known_face

[array([-1.78867411e-02,  4.32874784e-02,  1.31005883e-01, -4.88729402e-02,
        -1.12404667e-01, -4.06488590e-02,  3.86921018e-02, -8.68269354e-02,
         7.96081126e-02, -6.55392976e-03,  1.97670206e-01, -6.04967065e-02,
        -2.30242729e-01, -9.97099001e-03, -9.48795378e-02,  8.00418332e-02,
        -1.00199431e-01, -1.73904657e-01, -3.73390801e-02, -4.78330627e-02,
         3.81599441e-02,  3.07813622e-02,  1.23803387e-04,  3.45987752e-02,
        -1.10746488e-01, -2.61687547e-01, -1.18353494e-01, -9.84217599e-02,
        -1.55997630e-02, -1.11528665e-01,  2.65662633e-02,  1.32046372e-01,
        -1.93042561e-01, -1.11151144e-01,  2.73719281e-02,  4.26536612e-02,
        -6.32073507e-02, -1.20941259e-01,  1.92594826e-01,  7.67312124e-02,
        -1.19672239e-01,  7.32979178e-02, -1.37590617e-02,  3.58292043e-01,
         1.21472716e-01,  2.80619469e-02,  1.64976269e-02, -6.60775304e-02,
         1.10417128e-01, -2.73196965e-01,  4.42938060e-02,  1.67043731e-01,
         6.7

In [82]:
txt_sp = pyttsx3.init()
voices = txt_sp.getProperty('voices')

print("Voices available:", len(voices))

# ✅ safe voice selection (DO NOT assume index 1 exists)
if len(voices) > 1:
    txt_sp.setProperty('voice', voices[1].id)
else:
    txt_sp.setProperty('voice', voices[0].id)

txt_sp.setProperty('volume', 1.0)

Voices available: 3


In [83]:
import threading

def speak(engine, text):
    engine.say(text)
    engine.runAndWait()

video = cv2.VideoCapture(0)
last_spoken_time = {}
SPEAK_COOLDOWN = 3

while True:
    success, img = video.read()
    if not success:
        break
    img_1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_loc_v = fr.face_locations(img_1)
    face_enc_v = fr.face_encodings(img_1, face_loc_v)

    for enc, loc in zip(face_enc_v, face_loc_v):
        matches = fr.compare_faces(encoded_known_face, enc)
        face_dis = fr.face_distance(encoded_known_face, enc)
        match_index = np.argmin(face_dis)
        name = 'unknown'
        if matches[match_index] == True:
            name = class_names[match_index]

        current_time = time.time()
        last_time = last_spoken_time.get(name, 0)

        if current_time - last_time > SPEAK_COOLDOWN:
            print(name)
            # ✅ Run speech in a background thread — won't block the video
            t = threading.Thread(target=speak, args=(txt_sp, name))
            t.daemon = True
            t.start()
            last_spoken_time[name] = current_time

        y1, x2, y2, x1 = loc
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(img, name, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Frame', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

unknown
unknown
unknown
Mohanlal_Viswanathan_BNC
unknown
Mohanlal_Viswanathan_BNC
unknown
